# Installation de HypergraphPercol

In [1]:
%%bash
# Colab: paquets système pour CGAL/TBB/CMake
apt-get update -qq
apt-get install -y -qq build-essential cmake libcgal-dev libtbb-dev libtbbmalloc2 \
    libgmp-dev libmpfr-dev libeigen3-dev

Selecting previously unselected package libgmpxx4ldbl:amd64.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../libgmpxx4ldbl_2%3a6.2.1+dfsg-3ubuntu1_amd64.deb ...
Unpacking libgmpxx4ldbl:amd64 (2:6.2.1+dfsg-3ubuntu1) ...
Selecting previously unselected package libgmp-dev:amd64.
Preparing to unpack .../libgmp-dev_2%3a6.2.1+dfsg-3ubuntu1_amd64.deb ...
Unpacking libgmp-dev:amd64 (2:6.2.1+dfsg-3ubuntu1) ...
Selecting previously unselected package libmpfr-dev:amd64.
Preparing to unpack .../libmpfr-dev_4.1.0-3build3_amd64.deb ...
Unpacking libmpfr-dev:amd64 (4.1.0-3build3) ...
Selecting previously unselected package libcgal-dev:amd64.
Preparing to unpack .../libcgal-dev_5.4-1_amd64.deb ...
Unpacking libcgal-dev:amd64 (5.4-1) ...
Selecting previously unselected package libeigen3-dev.
Preparing to unpack .../libeigen3-dev_3.4.0-2ubuntu2_all.deb ...
Unpacking libeigen3-dev (3.4.0-2ubuntu2) ...
Setting up libgmpxx4ldbl:amd64 (2:6.2.1+dfsg-3ubuntu1) 

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!pip install -q --upgrade pip setuptools wheel Cython cmake jedi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 93.0 MB/s eta 0:00:00


In [ ]:
%%bash
set -euo pipefail
WORKDIR="${HGP_WORKDIR:-/content}"
mkdir -p "${WORKDIR}"
cd "${WORKDIR}"
if [ -d HGP-clusterer ]; then
    git -C HGP-clusterer pull --ff-only
else
    git clone https://github.com/Ludwig-H/HGP-clusterer.git
fi
if [ -d cyminiball ]; then
    git -C cyminiball pull --ff-only
else
    git clone https://github.com/Ludwig-H/cyminiball.git
fi


In [ ]:
%%bash
set -euo pipefail
WORKDIR="${HGP_WORKDIR:-/content}"
mkdir -p "${WORKDIR}/wheels"
cd "${WORKDIR}/cyminiball"
python3 -m pip wheel --no-build-isolation --no-deps --wheel-dir="${WORKDIR}/wheels" .
python3 -m pip install --force-reinstall --no-deps --no-index --find-links="${WORKDIR}/wheels" cyminiball
# Le "--no-deps" indispensable pour que numpy ne se télécharge pas en version 2.3.4, créant des problèmes de compatibilité...


In [ ]:
%%bash
set -euo pipefail
WORKDIR="${HGP_WORKDIR:-/content}"
cd "${WORKDIR}/HGP-clusterer"
python3 scripts/setup_cgal.py


In [ ]:
%%bash
set -euo pipefail
WORKDIR="${HGP_WORKDIR:-/content}"
cd "${WORKDIR}/HGP-clusterer/CGALDelaunay"

projects=(
    EdgesCGALDelaunay2D
    EdgesCGALDelaunay3D
    EdgesCGALDelaunayND
    EdgesCGALWeightedDelaunay2D
    EdgesCGALWeightedDelaunay3D
    EdgesCGALWeightedDelaunayND
)

for project in "${projects[@]}"; do
    cmake -S "${project}" -B "${project}/build" -DCMAKE_BUILD_TYPE=Release
    cmake --build "${project}/build" --config Release
    cmake --install "${project}/build" --prefix "${WORKDIR}/HGP-clusterer"
done


In [ ]:
%%bash
set -euo pipefail
WORKDIR="${HGP_WORKDIR:-/content}"
cd "${WORKDIR}/HGP-clusterer"
python3 -m pip install -v --no-deps .


In [ ]:
import os

workdir = os.environ.get("HGP_WORKDIR", "/content")
repo_root = os.path.join(workdir, "HGP-clusterer")
os.environ["CGALDELAUNAY_ROOT"] = os.path.join(repo_root, "CGALDelaunay")

import hgp_clusterer

In [ ]:
import hgp_clusterer

# List all attributes (functions, classes, variables) in hgp_clusterer
print("Functions and attributes in hgp_clusterer:")
for item in dir(hgp_clusterer):
    # if not item.startswith('_'): # Exclude private/special methods
    print(item)


In [ ]:
from hgp_clusterer import HypergraphPercol

You can also get help on a specific function, for example `HypergraphPercol` (which was used earlier in the notebook), by running `help(hgp_clusterer.HypergraphPercol)`.

In [ ]:
# Example: Get help on the HypergraphPercol function
# help(hgp_clusterer.HypergraphPercol)


# Application de HypergraphPercol aux *reads* Nanopore

In [ ]:
# @title Setup: dépendances et données
!pip -q install datasketch edlib numpy pandas scipy scikit-learn tqdm pyarrow

import os, sys, re, math, random, itertools, json
from pathlib import Path

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True, parents=True)

# Raw URLs du dépôt Microsoft (dataset seul)
CENTERS_URL  = "https://raw.githubusercontent.com/microsoft/clustered-nanopore-reads-dataset/main/Centers.txt"
CLUSTERS_URL = "https://raw.githubusercontent.com/microsoft/clustered-nanopore-reads-dataset/main/Clusters.txt"

def download(url, dst):
    import urllib.request
    if not Path(dst).exists():
        print(f"Téléchargement {url} -> {dst}")
        urllib.request.urlretrieve(url, dst)
    else:
        print(f"Déjà présent: {dst}")

download(CENTERS_URL,  DATA_DIR/"Centers.txt")
download(CLUSTERS_URL, DATA_DIR/"Clusters.txt")


In [ ]:
# @title Parsing des fichiers { vertical-output: true }

from collections import defaultdict, Counter
from typing import List, Tuple, Dict, Iterable, Optional
import numpy as np
import pandas as pd

def parse_clusters(clusters_path: Path) -> Tuple[List[str], np.ndarray]:
    reads = []
    labels = []
    label  = -1
    with open(clusters_path, "r") as f:
        for raw in f:
            s = raw.strip()
            if not s:
                continue
            if s.startswith("="):           # lignes de séparation "===="
                label += 1
                continue
            reads.append(s)
            labels.append(label)
    return reads, np.array(labels, dtype=np.int32)

reads, labels = parse_clusters(DATA_DIR/"Clusters.txt")
n_reads = len(reads)
n_clusters_seen = labels.max() + 1
print(f"{n_reads} reads, {n_clusters_seen} clusters (certains peuvent être vides).")

# Option: charger Centers pour évaluation ultérieure
centers = [line.strip() for line in open(DATA_DIR/'Centers.txt') if line.strip()]
print(f"{len(centers)} centers de longueur attendue 110.")


In [ ]:
from typing import List, Tuple
from collections import Counter
import edlib, random, math

# ------------------------------
# Utilitaires d'alignement edlib
# ------------------------------

def _cigar_to_gapped(a: str, b: str, cigar: str) -> Tuple[str, str]:
    """
    Convertit un CIGAR edlib (=,X,I,D) en deux chaînes gappées de même longueur.
    '=' : match, 'X' : mismatch, 'I' : insertion dans la cible (gap côté a), 'D' : deletion (gap côté b).
    On tolère 'M' au cas où.
    """
    ia = ib = 0
    A, B = [], []
    num = 0

    for ch in cigar:
        if ch.isdigit():
            num = num * 10 + int(ch)
            continue
        cnt = num if num > 0 else 1
        num = 0

        if ch in ('=', 'M', 'X'):
            for _ in range(cnt):
                ca = a[ia] if ia < len(a) else '-'
                cb = b[ib] if ib < len(b) else '-'
                A.append(ca)
                B.append(cb)
                if ia < len(a): ia += 1
                if ib < len(b): ib += 1
        elif ch == 'I':
            for _ in range(cnt):
                A.append('-')
                cb = b[ib] if ib < len(b) else '-'
                B.append(cb)
                if ib < len(b): ib += 1
        elif ch == 'D':
            for _ in range(cnt):
                ca = a[ia] if ia < len(a) else '-'
                A.append(ca)
                B.append('-')
                if ia < len(a): ia += 1
        else:
            raise ValueError(f"Opérateur CIGAR non géré: {ch}")

    L = max(len(A), len(B))
    if len(A) < L: A.extend(['-'] * (L - len(A)))
    if len(B) < L: B.extend(['-'] * (L - len(B)))
    return ''.join(A), ''.join(B)


def global_align_edlib(a: str, b: str) -> Tuple[str, str]:
    """Alignement global simple via edlib (orientation imposée)."""
    res = edlib.align(a, b, mode="NW", task="path")
    A, B = _cigar_to_gapped(a, b, res["cigar"])
    return A, B


def global_align_edlib_best_oriented(a: str, b: str) -> Tuple[str, str]:
    """
    Alignement global en choisissant automatiquement l’orientation de b (directe ou renversée)
    qui minimise la distance d’édition.
    """
    res_dir = edlib.align(a, b, mode="NW", task="path")
    d_dir = res_dir["editDistance"]

    b_rev = b[::-1]
    res_rev = edlib.align(a, b_rev, mode="NW", task="path")
    d_rev = res_rev["editDistance"]

    if d_rev < d_dir:
        return _cigar_to_gapped(a, b_rev, res_rev["cigar"])
    else:
        return _cigar_to_gapped(a, b, res_dir["cigar"])


def _bidir_edit(a: str, b: str) -> Tuple[int, bool]:
    """
    Distance d’édition globale min{ dist(a,b), dist(a,rev(b)) }.
    Retourne (meilleure_distance, used_reversed).
    """
    d = edlib.align(a, b, mode="NW", task="distance")["editDistance"]
    d_rev = edlib.align(a, b[::-1], mode="NW", task="distance")["editDistance"]
    if d_rev < d:
        return d_rev, True
    return d, False


def norm_edit(a: str, b: str) -> float:
    """
    Distance d'édition normalisée, en autorisant le renversement.
    """
    best, _ = _bidir_edit(a, b)
    return best / max(1, max(len(a), len(b)))


# -----------------------------------
# Sélection du méduloïde (médoïde lite) — bi-orientation
# -----------------------------------

def medoid_index(read_ids: List[int], reads: List[str], sample_size: int = 200, band: int = 20) -> int:
    """
    Retourne l'indice (dans read_ids) du read qui minimise la somme des distances aux autres
    (sur un échantillon), en autorisant le renversement au niveau pairwise.
    Paramètre 'band' ignoré: on utilise uniquement edlib en global.
    """
    if len(read_ids) == 1:
        return 0

    sample = read_ids if len(read_ids) <= sample_size else random.sample(read_ids, sample_size)

    best_tot, best_r = float('inf'), None
    for r in sample:
        tot = 0.0
        ar = reads[r]
        for s in sample:
            if s == r:
                continue
            # distance bi-orientée (direct vs renversée)
            d_rs, _ = _bidir_edit(ar, reads[s])
            tot += d_rs / max(1, max(len(ar), len(reads[s])))
        if tot < best_tot:
            best_tot, best_r = tot, r

    return read_ids.index(best_r)


# ----------------------------------------------------
# Ajustement à longueur 110 (inchangé)
# ----------------------------------------------------

def adjust_to_length_110(draft: str, aligned_reads: List[str], aligned_draft: List[str], target_len: int = 110) -> str:
    if not aligned_draft or not aligned_reads:
        return draft[:target_len].ljust(target_len, 'A')

    L = max(max(len(s) for s in aligned_draft), max(len(s) for s in aligned_reads))

    col_stats = []
    for c in range(L):
        draft_chars = [ad[c] if c < len(ad) else '-' for ad in aligned_draft]
        read_chars  = [ar[c] if c < len(ar) else '-' for ar in aligned_reads]

        ins  = sum(1 for dch, rch in zip(draft_chars, read_chars) if dch == '-' and rch != '-')
        dele = sum(1 for dch, rch in zip(draft_chars, read_chars) if dch != '-' and rch == '-')

        cnt = Counter([rch for rch in read_chars if rch != '-'])
        if cnt:
            maj_base, maj_count = cnt.most_common(1)[0]
            tot = sum(cnt.values())
            probs = [v / max(1, tot) for v in cnt.values()]
            entropy = -sum(p * math.log2(p + 1e-12) for p in probs)
        else:
            maj_base, maj_count, entropy = ('A', 0, 0.0)

        col_stats.append({"ins": ins, "del": dele, "maj": maj_base, "majc": maj_count, "entropy": entropy})

    ref_aligned_draft = aligned_draft[0] if aligned_draft else "-" * L
    if len(ref_aligned_draft) < L:
        ref_aligned_draft = ref_aligned_draft + "-" * (L - len(ref_aligned_draft))

    cols = []
    thr_major = max(2, len(aligned_reads) // 3)
    for c in range(L):
        if ref_aligned_draft[c] == '-':
            cols.append(col_stats[c]["maj"] if col_stats[c]["majc"] > 0 else 'A')
        else:
            base = ref_aligned_draft[c]
            if col_stats[c]["majc"] >= thr_major:
                base = col_stats[c]["maj"]
            cols.append(base)

    if len(cols) > target_len:
        order = sorted(range(len(cols)), key=lambda c: (col_stats[c]["del"], col_stats[c]["entropy"]), reverse=True)
        to_drop = set(order[:len(cols) - target_len])
        cols = [cols[c] for c in range(len(cols)) if c not in to_drop]
    elif len(cols) < target_len:
        need = target_len - len(cols)
        order = sorted(range(len(cols)), key=lambda c: col_stats[c]["ins"], reverse=True)
        ins_positions = order[:max(1, min(need, len(order)))]
        pos_iter = 0
        while need > 0:
            c = ins_positions[pos_iter % len(ins_positions)]
            cols.insert(c, col_stats[c]["maj"] if col_stats[c]["majc"] > 0 else 'A')
            need -= 1
            pos_iter += 1

    if len(cols) != target_len:
        cols = (cols[:target_len] if len(cols) > target_len else cols + ['A'] * (target_len - len(cols)))
    return ''.join(cols)


# -----------------------------------------
# Calcul du "center" de cluster via consensus
# -----------------------------------------

def compute_cluster_center(read_ids: List[int], reads: List[str],
                           target_len: int = 110, band: int = 20, max_reads_align: int = 400) -> str:
    """
    Identique à ta version, mais les alignements pairwise se font en orientation optimale
    (directe ou renversée) pour éviter de polluer le consensus.
    """
    if not read_ids:
        return ""

    ids_for_align = read_ids if len(read_ids) <= max_reads_align else random.sample(read_ids, max_reads_align)

    # médoïde (bi-orientation dans medoid_index)
    midx_in_ids = medoid_index(ids_for_align, reads, sample_size=min(200, len(ids_for_align)), band=band)
    medoid_read_id = ids_for_align[midx_in_ids]
    draft = reads[medoid_read_id]

    # alignement des reads sur le draft (orientation optimale)
    aligned_draft, aligned_reads = [], []
    for rid in ids_for_align:
        A, B = global_align_edlib_best_oriented(draft, reads[rid])
        L = max(len(A), len(B))
        if len(A) < L: A += '-' * (L - len(A))
        if len(B) < L: B += '-' * (L - len(B))
        aligned_draft.append(A)
        aligned_reads.append(B)

    center = adjust_to_length_110(draft, aligned_reads, aligned_draft, target_len=target_len)

    # raffinement: réaligner sur le center (orientation optimale)
    aligned_center, aligned_reads2 = [], []
    for rid in ids_for_align:
        A, B = global_align_edlib_best_oriented(center, reads[rid])
        L = max(len(A), len(B))
        if len(A) < L: A += '-' * (L - len(A))
        if len(B) < L: B += '-' * (L - len(B))
        aligned_center.append(A)
        aligned_reads2.append(B)

    L = max(len(s) for s in aligned_center) if aligned_center else len(center)
    aligned_center = [s.ljust(L, '-') for s in aligned_center]
    aligned_reads2 = [s.ljust(L, '-') for s in aligned_reads2]

    cols = []
    for c in range(L):
        cnt = Counter([ar[c] for ar in aligned_reads2 if c < len(ar) and ar[c] != '-'])
        if cnt:
            base = cnt.most_common(1)[0][0]
        else:
            base = center[c] if c < len(center) and center[c] != '-' else 'A'
        cols.append(base)

    final = ''.join([x for x in cols if x != '-'])
    if len(final) != target_len:
        final = (final[:target_len] if len(final) > target_len else final + 'A' * (target_len - len(final)))
    return final


In [ ]:
# @title LSH (direct + renversé) + Graphe sparse top-K (bi-orientation) { vertical-output: true }

# Installs (au cas où)
!pip -q install edlib datasketch joblib tqdm tqdm-joblib

from datasketch import MinHash, MinHashLSH
from joblib import Parallel, delayed, cpu_count
from tqdm.auto import tqdm
try:
    from tqdm_joblib import tqdm_joblib
except ImportError:
    tqdm_joblib = None

import edlib, random, numpy as np

# =========================
# Hyperparamètres
# =========================
NUM_PERM      = 128    # taille du sketch MinHash
KMER          = 15     # k-mer size pour ~110 nt
LSH_THRESHOLD = 0.45   # seuil Jaccard approx -> candidats
SEED          = 1337

# Graphe
TOP_K        = 25        # voisins conservés par nœud (après LSH)
MAX_EDGES    = None      # limite dure sur le nb d'arêtes (None = pas de limite)
NORMALIZE_BY = "maxlen"  # "maxlen" ou "meanlen"

random.seed(SEED)

# =========================
# K-mers et MinHash helpers
# =========================
def kmers(seq: str, k: int):
    L = len(seq)
    if L < k:
        return []
    return [seq[i:i+k] for i in range(L - k + 1)]

def _sketch_from_kmers(kmer_iter, num_perm=NUM_PERM, seed=SEED) -> MinHash:
    m = MinHash(num_perm=num_perm, seed=seed)
    for km in kmer_iter:
        m.update(km.encode("utf8"))
    return m

def _sketch_direct(seq: str, k=KMER, num_perm=NUM_PERM) -> MinHash:
    return _sketch_from_kmers(kmers(seq, k), num_perm=num_perm, seed=SEED)

def _sketch_reversed(seq: str, k=KMER, num_perm=NUM_PERM) -> MinHash:
    rseq = seq[::-1]
    return _sketch_from_kmers(kmers(rseq, k), num_perm=num_perm, seed=SEED)

def build_two_sketches(reads, k=KMER, num_perm=NUM_PERM, n_jobs=None, batch_size=64):
    """
    Construit deux ensembles de MinHash:
      - direct_sketches[i]   = sketch du read i
      - reversed_sketches[i] = sketch du read i renversé
    """
    if n_jobs is None:
        n_jobs = cpu_count()

    if n_jobs == 1 or tqdm_joblib is None:
        direct = [ _sketch_direct(s, k=k, num_perm=num_perm)   for s in tqdm(reads, desc="MinHash direct",   unit="read") ]
        rev    = [ _sketch_reversed(s, k=k, num_perm=num_perm) for s in tqdm(reads, desc="MinHash reversed", unit="read") ]
    else:
        with tqdm_joblib(tqdm(total=len(reads), desc="MinHash direct (parallel)", unit="read")):
            direct = Parallel(n_jobs=n_jobs, prefer="processes", batch_size=batch_size)(
                delayed(_sketch_direct)(s, k=k, num_perm=num_perm) for s in reads
            )
        with tqdm_joblib(tqdm(total=len(reads), desc="MinHash reversed (parallel)", unit="read")):
            rev = Parallel(n_jobs=n_jobs, prefer="processes", batch_size=batch_size)(
                delayed(_sketch_reversed)(s, k=k, num_perm=num_perm) for s in reads
            )
    return direct, rev

def build_lsh_direct_and_reversed(direct_sketches, reversed_sketches,
                                  threshold=LSH_THRESHOLD, num_perm=NUM_PERM):
    """
    Construit un index LSH contenant deux entrées par read:
      - clé (i, 0) -> sketch direct[i]
      - clé (i, 1) -> sketch reversed[i]
    """
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    for i, m in enumerate(direct_sketches):
        lsh.insert((i, 0), m)
    for i, m in enumerate(reversed_sketches):
        lsh.insert((i, 1), m)
    return lsh

def lsh_query_both_for_index(lsh, direct_sketches, reversed_sketches, idx: int):
    """
    Requête l'index pour un read existant (par son index) en bi-orientation.
    Retourne un set d'identifiants de reads voisins (sans orientation), sans idx lui-même.
    """
    cand = set()
    for key in lsh.query(direct_sketches[idx]):
        cand.add(key[0])
    for key in lsh.query(reversed_sketches[idx]):
        cand.add(key[0])
    cand.discard(idx)
    return cand

# =========================
# Distances d'édition
# =========================
def best_oriented_edit(a: str, b: str) -> int:
    """
    Distance d’édition globale min{ dist(a,b), dist(a,rev(b)) } via edlib (NW).
    """
    d1 = edlib.align(a, b,     mode="NW", task="distance")["editDistance"]
    d2 = edlib.align(a, b[::-1], mode="NW", task="distance")["editDistance"]
    return d1 if d1 <= d2 else d2

def norm_edit(a: str, b: str, how=NORMALIZE_BY) -> float:
    """
    Distance normalisée d’édition bi-orientation.
    how = 'maxlen' ou 'meanlen'
    """
    d = best_oriented_edit(a, b)
    if how == "meanlen":
        denom = (len(a) + len(b)) / 2.0
    else:
        denom = max(len(a), len(b))
    return d / max(1.0, denom)

# =========================
# Build sketches + LSH
# =========================
# 'reads' doit exister: liste[str] des séquences
direct_sketches, reversed_sketches = build_two_sketches(
    reads, k=KMER, num_perm=NUM_PERM, n_jobs=None, batch_size=64
)
lsh = build_lsh_direct_and_reversed(direct_sketches, reversed_sketches,
                                    threshold=LSH_THRESHOLD, num_perm=NUM_PERM)

print("Index LSH construit (direct + renversé).")




In [ ]:
# =========================
# Construction du graphe
# =========================
edges = []   # liste de tuples (i, j, w_edit_norm, jacc_max)
edge_count = 0

# utilitaire: score Jaccard approx bi-orientation pour prioriser
def jaccard_biorient(i: int, j: int) -> float:
    # on prend le max entre les 4 combinaisons plausibles
    sdd = direct_sketches[i].jaccard(direct_sketches[j])
    sdr = direct_sketches[i].jaccard(reversed_sketches[j])
    srd = reversed_sketches[i].jaccard(direct_sketches[j])
    srr = reversed_sketches[i].jaccard(reversed_sketches[j])
    return max(sdd, sdr, srd, srr)

with tqdm(total=len(reads), desc="Voisins + distances", unit="node") as pbar:
    for i in range(len(reads)):
        # Candidats bi-orientation via LSH
        cand = list(lsh_query_both_for_index(lsh, direct_sketches, reversed_sketches, i))
        if not cand:
            pbar.update(1)
            continue

        # Scoring approx (Jaccard) pour prioriser les plus proches
        scored = [(j, jaccard_biorient(i, j)) for j in cand]
        scored.sort(key=lambda t: t[1], reverse=True)
        keep = scored[:TOP_K] if TOP_K and TOP_K > 0 else scored

        # Rerank local par vraie distance d'édition bi-orientation
        for j, jac in keep:
            # if j <= i:
            #     # on ne crée l’arête (i,j) qu’une seule fois (graphe non orienté)
            #     continue
            d_norm = norm_edit(reads[i], reads[j], how=NORMALIZE_BY)
            edges.append((min(i,j), max(i, j), float(d_norm), float(jac)))
            edge_count += 1
            if MAX_EDGES and edge_count >= MAX_EDGES:
                break

        if MAX_EDGES and edge_count >= MAX_EDGES:
            break

        pbar.update(1)



print(f"{len(edges)} arêtes construites (non orientées, parallèle, mémoire bornée).")

# HypergraphPercol

In [ ]:
!pip install gudhi

M = [(min(i,j),max(i,j),d) for i,j,d,_ in edges]
K = 1
min_cluster_size = 3
metric = "sparse"
expZ=0.5
verbeux = True

labels_pred = HypergraphPercol(M=M,
                               K=K,
                               expZ=expZ,
                               min_cluster_size=min_cluster_size,
                               metric=metric,
                               verbeux=verbeux)

In [ ]:

pred_clusters = sorted([int(c) for c in set(labels_pred) if c != -1])
print("Nb clusters prédits (sans -1):", len(pred_clusters))

cluster_to_readids = defaultdict(list)
for rid, c in enumerate(labels_pred):
    c = int(c)
    if c != -1:
        cluster_to_readids[c].append(rid)

pred_centers = {}
for c in tqdm(pred_clusters, desc="Centers by cluster"):
    c = int(c)
    read_ids = cluster_to_readids[c]
    pred_centers[c] = compute_cluster_center(read_ids, reads, target_len=110, band=20, max_reads_align=400)

rows = [{"pred_cluster": c, "pred_center": s} for c, s in pred_centers.items()]
# pd.DataFrame(rows).to_csv(CENTERS_CSV, index=False)
# print("Saved predicted centers:", CENTERS_CSV)


In [ ]:
np.max(pred_clusters)
cluster_to_readids
pred_centers

In [ ]:
# %%
# ===========================
# Metrics sur centers (edlib)
# Parallelisé avec joblib + tqdm_joblib
# Matching et stats bi-orientation (direct / reversed)
# ===========================
from typing import Dict, List, Tuple
import numpy as np
import pandas as pd
import math
import edlib

from joblib import Parallel, delayed
from tqdm.auto import tqdm
from tqdm_joblib import tqdm_joblib


# ----------------------------
# Distances via edlib
# ----------------------------
def _ed_mode(a: str, b: str, mode: str = "NW") -> int:
    """Distance d’édition via edlib pour un mode donné: 'NW' (global) ou 'HW' (semi-global)."""
    return edlib.align(a, b, mode=mode, task="distance")["editDistance"]

def _best_bidorient(a: str, b: str, mode: str = "NW") -> Tuple[int, bool, int, int]:
    """
    Retourne (best_dist, used_reversed, dist_direct, dist_reversed)
    où best_dist = min(dist(a,b), dist(a,b[::-1])) sur le mode edlib demandé.
    """
    d_dir = _ed_mode(a, b, mode=mode)
    d_rev = _ed_mode(a, b[::-1], mode=mode)
    if d_rev < d_dir:
        return d_rev, True, d_dir, d_rev
    else:
        return d_dir, False, d_dir, d_rev


# ----------------------------------------------------
# Passes parallèles: meilleur voisin de l'autre côté
#   Matching fait sur distance globale (NW) en bi-orientation
# ----------------------------------------------------
def _best_true_for_one_pred(pseq: str, centers: List[str]) -> Tuple[int, int]:
    """
    Calcule le meilleur true center (index) pour un center prédit donné,
    en minimisant la distance d'édition GLOBALE (NW) sur {direct, reversed}.
    Retourne (true_idx, best_dist_NW).
    """
    best_i, best_d = None, math.inf
    for i, tseq in enumerate(centers):
        d, _, _, _ = _best_bidorient(pseq, tseq, mode="NW")
        if d < best_d:
            best_d, best_i = d, i
    return best_i, int(best_d)


def _best_pred_for_one_true(tseq: str, pred_items: List[Tuple[int, str]]) -> Tuple[int, int]:
    """
    Symétrique: pour un true center donné, retourne (pred_key, best_dist_NW) en bi-orientation.
    """
    best_pk, best_d = None, math.inf
    for pk, pseq in pred_items:
        d, _, _, _ = _best_bidorient(pseq, tseq, mode="NW")
        if d < best_d:
            best_d, best_pk = d, pk
    return best_pk, int(best_d)


# ----------------------------------------------------
# Matching glouton 1-1 par distance croissante (NW)
# ----------------------------------------------------
def _greedy_match_from_candidates(candidates: List[Tuple[int, int, int]],
                                  n_true: int,
                                  threshold: int = None) -> List[Tuple[int, int, int]]:
    """
    candidates: liste de (dist_NW, pred_key, true_idx)
    Retourne une liste de paires (true_idx, pred_key, dist_NW) appariées 1-1.
    """
    candidates.sort(key=lambda t: t[0])  # dist croissante
    used_true, used_pred = set(), set()
    pairs = []
    for d, pk, ti in candidates:
        if threshold is not None and d > threshold:
            continue
        if pk in used_pred or ti in used_true:
            continue
        used_pred.add(pk)
        used_true.add(ti)
        pairs.append((ti, pk, d))
        if len(used_true) == n_true:
            break
    return pairs


# ----------------------------------------------------
# Fonction principale
# ----------------------------------------------------
def compute_center_metrics_parallel(centers: List[str],
                                    pred_centers: Dict[int, str],
                                    THRESHOLD_PROX: int = None,
                                    n_jobs: int = -1) -> Tuple[dict, pd.DataFrame, pd.DataFrame]:
    """
    Calcule des métriques de pertinence des centers prédits, en parallèle.
    Matching: distance d'édition GLOBALE (NW) avec inversion autorisée.
    Statistiques: sur les paires retenues, calcule aussi les distances HW (semi-global),
                  toujours en bi-orientation.
    Sorties:
      - summary: dict de métriques globales
      - df_pairs: DataFrame détaillée des appariements (d_NW/HW direct/rev, orientation choisie)
      - df_mintrue: min distance NW (bi-orientation) pour chaque true vers tous les pred
    """

    n_true = len(centers)
    pred_items = sorted(pred_centers.items())  # liste [(pred_key, seq), ...]
    n_pred = len(pred_items)

    # Cas trivial
    if n_true == 0 and n_pred == 0:
        return {
            "n_true_centers": 0, "n_pred_centers": 0,
            "classes_retrouvees": 0, "precision_classes": 0.0,
            "recall_classes": 0.0, "f1_classes": 0.0, "classes_en_trop": 0,
            "avg_editdist_on_matched_NW": None, "median_editdist_on_matched_NW": None,
            "p90_editdist_on_matched_NW": None, "max_editdist_on_matched_NW": None,
            "exact_match_rate_on_matched_NW": 0.0,
            "avg_editdist_on_matched_HW": None, "median_editdist_on_matched_HW": None,
            "p90_editdist_on_matched_HW": None, "max_editdist_on_matched_HW": None,
            "exact_match_rate_on_matched_HW": 0.0,
            "count_dist_0_NW": 0, "count_dist_1_NW": 0, "count_dist_2_NW": 0, "count_dist_3_NW": 0,
            "count_dist_0_HW": 0, "count_dist_1_HW": 0, "count_dist_2_HW": 0, "count_dist_3_HW": 0,
        }, pd.DataFrame(), pd.DataFrame()

    # -------------------------------
    # Passe 1: pour chaque prédict, meilleur TRUE (NW bi-orient)
    # -------------------------------
    with tqdm_joblib(tqdm(desc="Best TRUE for each PRED (NW, bi-orient)", total=n_pred)) as _:
        best_true_list = Parallel(n_jobs=n_jobs, backend="loky")(
            delayed(_best_true_for_one_pred)(pseq, centers) for _, pseq in pred_items
        )
    best_true_for_pred_map = {pk: res for (pk, _), res in zip(pred_items, best_true_list)}
    min_pred_over_trues_NW = np.array([d for (_, d) in best_true_list], dtype=float)  # par pred

    # Fabrication des candidats (dist_NW, pred_key, true_idx) à partir de la passe 1
    candidates = [(best_true_for_pred_map[pk][1], pk, best_true_for_pred_map[pk][0])
                  for pk, _ in pred_items if best_true_for_pred_map[pk][0] is not None]

    # -------------------------------
    # Passe 2: pour chaque TRUE, meilleur PRED (NW bi-orient)
    # -------------------------------
    with tqdm_joblib(tqdm(desc="Best PRED for each TRUE (NW, bi-orient)", total=n_true)) as _:
        best_pred_list = Parallel(n_jobs=n_jobs, backend="loky")(
            delayed(_best_pred_for_one_true)(tseq, pred_items) for tseq in centers
        )
    best_pred_for_true_map = {i: (pk, d) for i, (pk, d) in enumerate(best_pred_list)}
    min_true_over_preds_NW = np.array([d for (_, d) in best_pred_list], dtype=float)  # par true

    # -------------------------------
    # Matching glouton final (NW)
    # -------------------------------
    pairs = _greedy_match_from_candidates(candidates, n_true, threshold=THRESHOLD_PROX)
    classes_retrouvees = len(pairs)
    precision_classes = classes_retrouvees / max(1, n_pred)
    recall_classes = classes_retrouvees / max(1, n_true)
    f1_classes = (2 * precision_classes * recall_classes) / max(1e-12, precision_classes + recall_classes)
    classes_en_trop = max(0, n_pred - classes_retrouvees)

    # -------------------------------
    # Distances détaillées pour paires: NW et HW, direct vs reversed
    # -------------------------------
    def _pair_stats(ti: int, pk: int) -> dict:
        tseq = centers[ti]
        pseq = dict(pred_items)[pk]

        # NW bi-orientation
        best_NW, used_rev_NW, ddir_NW, drev_NW = _best_bidorient(pseq, tseq, mode="NW")
        # HW bi-orientation
        best_HW, used_rev_HW, ddir_HW, drev_HW = _best_bidorient(pseq, tseq, mode="HW")

        return {
            "true_idx": int(ti),
            "pred_key": int(pk),
            "edit_distance_NW": int(best_NW),
            "edit_distance_HW": int(best_HW),
            "direct_NW": int(ddir_NW),
            "reversed_NW": int(drev_NW),
            "direct_HW": int(ddir_HW),
            "reversed_HW": int(drev_HW),
            "used_reversed_for_best_NW": bool(used_rev_NW),
            "used_reversed_for_best_HW": bool(used_rev_HW),
            "norm_edit_distance_NW": float(best_NW / max(1, len(tseq))),
            "norm_edit_distance_HW": float(best_HW / max(1, len(tseq))),
        }

    if pairs:
        with tqdm_joblib(tqdm(desc="Pair stats (NW & HW, bi-orient)", total=len(pairs))) as _:
            pair_rows = Parallel(n_jobs=n_jobs, backend="loky")(
                delayed(_pair_stats)(ti, pk) for (ti, pk, _) in pairs
            )
    else:
        pair_rows = []

    df_pairs = pd.DataFrame(pair_rows).sort_values(["edit_distance_NW", "true_idx"]) if pair_rows else pd.DataFrame(
        columns=["true_idx","pred_key","edit_distance_NW","edit_distance_HW","direct_NW","reversed_NW","direct_HW","reversed_HW","used_reversed_for_best_NW","used_reversed_for_best_HW","norm_edit_distance_NW","norm_edit_distance_HW"]
    )

    # -------------------------------
    # Agrégations et compteurs utiles
    # -------------------------------
    # Stats sur NW
    dists_NW = df_pairs["edit_distance_NW"].to_numpy() if not df_pairs.empty else np.array([], dtype=float)
    if dists_NW.size:
        avg_NW = float(dists_NW.mean()); med_NW = float(np.median(dists_NW))
        p90_NW = float(np.percentile(dists_NW, 90)); max_NW = float(dists_NW.max())
        exact_rate_NW = float((dists_NW == 0).mean())
        c0_NW = int((dists_NW == 0).sum())
        c1_NW = int((dists_NW == 1).sum())
        c2_NW = int((dists_NW == 2).sum())
        c3_NW = int((dists_NW == 3).sum())
    else:
        avg_NW = med_NW = p90_NW = max_NW = None
        exact_rate_NW = 0.0
        c0_NW = c1_NW = c2_NW = c3_NW = 0

    # Stats sur HW
    dists_HW = df_pairs["edit_distance_HW"].to_numpy() if not df_pairs.empty else np.array([], dtype=float)
    if dists_HW.size:
        avg_HW = float(dists_HW.mean()); med_HW = float(np.median(dists_HW))
        p90_HW = float(np.percentile(dists_HW, 90)); max_HW = float(dists_HW.max())
        exact_rate_HW = float((dists_HW == 0).mean())
        c0_HW = int((dists_HW == 0).sum())
        c1_HW = int((dists_HW == 1).sum())
        c2_HW = int((dists_HW == 2).sum())
        c3_HW = int((dists_HW == 3).sum())
    else:
        avg_HW = med_HW = p90_HW = max_HW = None
        exact_rate_HW = 0.0
        c0_HW = c1_HW = c2_HW = c3_HW = 0

    # Couverture TRUE -> PRED (NW bi-orientation)
    seq_len = len(centers[0]) if n_true else (len(next(iter(pred_centers.values()))) if n_pred else 110)
    df_mintrue = pd.DataFrame({
        "true_idx": np.arange(n_true, dtype=int),
        "min_edit_distance_to_any_pred_NW": min_true_over_preds_NW,
        "min_norm_edit_distance_to_any_pred_NW": (min_true_over_preds_NW / max(1, seq_len))
    })
    finite_true = np.isfinite(df_mintrue["min_edit_distance_to_any_pred_NW"].values)
    if finite_true.any():
        cover_mean = float(df_mintrue.loc[finite_true, "min_edit_distance_to_any_pred_NW"].mean())
        cover_med  = float(df_mintrue.loc[finite_true, "min_edit_distance_to_any_pred_NW"].median())
        cover_p90  = float(np.percentile(df_mintrue.loc[finite_true, "min_edit_distance_to_any_pred_NW"], 90))
        covered_rate = (float((df_mintrue["min_edit_distance_to_any_pred_NW"] <= THRESHOLD_PROX).mean())
                        if THRESHOLD_PROX is not None else None)
    else:
        cover_mean = cover_med = cover_p90 = covered_rate = None

    # Spécificité PRED -> TRUE (NW bi-orientation)
    finite_pred = np.isfinite(min_pred_over_trues_NW)
    if finite_pred.any():
        spec_mean = float(min_pred_over_trues_NW[finite_pred].mean())
        spec_med  = float(np.median(min_pred_over_trues_NW[finite_pred]))
        spec_p90  = float(np.percentile(min_pred_over_trues_NW[finite_pred], 90))
        specific_rate = (float((min_pred_over_trues_NW[finite_pred] <= THRESHOLD_PROX).mean())
                         if THRESHOLD_PROX is not None else None)
    else:
        spec_mean = spec_med = spec_p90 = specific_rate = None

    # Scores sous seuil (si fourni), sur les paires retenues, pour NW
    if THRESHOLD_PROX is not None and dists_NW.size:
        pairs_within_thr = int(np.sum(dists_NW <= THRESHOLD_PROX))
        precision_at_thr = pairs_within_thr / max(1, n_pred)
        recall_at_thr    = pairs_within_thr / max(1, n_true)
        f1_at_thr        = (2 * precision_at_thr * recall_at_thr) / max(1e-12, precision_at_thr + recall_at_thr)
    else:
        pairs_within_thr = precision_at_thr = recall_at_thr = f1_at_thr = None

    summary = {
        # Comptage
        "n_true_centers": int(n_true),
        "n_pred_centers": int(n_pred),

        # Matching glouton (NW bi-orient)
        "classes_retrouvees": int(classes_retrouvees),
        "precision_classes": float(precision_classes),
        "recall_classes": float(recall_classes),
        "f1_classes": float(f1_classes),
        "classes_en_trop": int(classes_en_trop),

        # Distances sur paires (NW)
        "avg_editdist_on_matched_NW": avg_NW,
        "median_editdist_on_matched_NW": med_NW,
        "p90_editdist_on_matched_NW": p90_NW,
        "max_editdist_on_matched_NW": max_NW,
        "exact_match_rate_on_matched_NW": float(exact_rate_NW),
        "count_dist_0_NW": int(c0_NW),
        "count_dist_1_NW": int(c1_NW),
        "count_dist_2_NW": int(c2_NW),
        "count_dist_3_NW": int(c3_NW),

        # Distances sur paires (HW)
        "avg_editdist_on_matched_HW": avg_HW,
        "median_editdist_on_matched_HW": med_HW,
        "p90_editdist_on_matched_HW": p90_HW,
        "max_editdist_on_matched_HW": max_HW,
        "exact_match_rate_on_matched_HW": float(exact_rate_HW),
        "count_dist_0_HW": int(c0_HW),
        "count_dist_1_HW": int(c1_HW),
        "count_dist_2_HW": int(c2_HW),
        "count_dist_3_HW": int(c3_HW),

        # Couverture vrais → prédits (NW bi-orient)
        "coverage_true_min_mean_NW": cover_mean,
        "coverage_true_min_median_NW": cover_med,
        "coverage_true_min_p90_NW": cover_p90,
        "coverage_true_rate_below_threshold_NW": covered_rate,

        # Spécificité prédits → vrais (NW bi-orient)
        "specificity_pred_min_mean_NW": spec_mean,
        "specificity_pred_min_median_NW": spec_med,
        "specificity_pred_min_p90_NW": spec_p90,
        "specificity_pred_rate_below_threshold_NW": specific_rate,

        # Scores sous seuil (si fourni) sur paires NW
        "pairs_within_threshold_NW": pairs_within_thr,
        "precision_at_threshold_NW": precision_at_thr,
        "recall_at_threshold_NW": recall_at_thr,
        "f1_at_threshold_NW": f1_at_thr,
    }

    return summary, df_pairs, df_mintrue


# -----
# Exemple:
# -----
THRESHOLD_PROX = 50
summary, df_pairs, df_mintrue = compute_center_metrics_parallel(
    centers, pred_centers, THRESHOLD_PROX=THRESHOLD_PROX, n_jobs=-1
)
print(summary)
display(df_pairs.head(), df_mintrue.head())



# LSH_THRESHOLD = 0.45

K=1
reads clustered=181699
Clusters finaux : 7650
'classes_retrouvees': 7114 (edit distance <= 50)
'avg_editdist_on_matched': 24.56452066348046, 'median_editdist_on_matched': 24.0, 'p90_editdist_on_matched': 39.0,
'count_dist_0': 10, 'count_dist_1': 10, 'count_dist_2': 26, 'count_dist_3': 9

***

K=2
reads clustered=128920
Clusters finaux : 13053
'classes_retrouvees': 7378 (edit distance <= 50)
'avg_editdist_on_matched': 17.950799674708595, 'median_editdist_on_matched': 16.0, 'p90_editdist_on_matched': 35.0,
'count_dist_0': 26, 'count_dist_1': 58, 'count_dist_2': 143, 'count_dist_3': 142,


***

K=3
reads clustered=81140
Clusters finaux : 13253
'classes_retrouvees': 7543 (edit distance <= 50)
'avg_editdist_on_matched': 14.73100888240753, 'median_editdist_on_matched': 11.0, 'p90_editdist_on_matched': 31.0,
'count_dist_0': 13, 'count_dist_1': 34, 'count_dist_2': 165, 'count_dist_3': 222

## $K=3$
[COMP-F:2] comp 5822/5822 done | cumul points labellisés=81140 | conflits=58244 | RSS=40.8 GiB
Clusters finaux : 13253 | bruit : 188568 | RSS=40.8 GiB

{'n_true_centers': 10000, 'n_pred_centers': 13253, 'classes_retrouvees': 7543, 'precision_classes': 0.5691541537765035, 'recall_classes': 0.7543, 'f1_classes': 0.6487765019567368, 'classes_en_trop': 5710, 'avg_editdist_on_matched_NW': 14.73100888240753, 'median_editdist_on_matched_NW': 11.0, 'p90_editdist_on_matched_NW': 31.0, 'max_editdist_on_matched_NW': 50.0, 'exact_match_rate_on_matched_NW': 0.0017234522073445579, 'count_dist_0_NW': 13, 'count_dist_1_NW': 34, 'count_dist_2_NW': 165, 'count_dist_3_NW': 222, 'avg_editdist_on_matched_HW': 12.537717088691503, 'median_editdist_on_matched_HW': 9.0, 'p90_editdist_on_matched_HW': 27.0, 'max_editdist_on_matched_HW': 50.0, 'exact_match_rate_on_matched_HW': 0.0017234522073445579, 'count_dist_0_HW': 13, 'count_dist_1_HW': 45, 'count_dist_2_HW': 216, 'count_dist_3_HW': 312, 'coverage_true_min_mean_NW': 20.7229, 'coverage_true_min_median_NW': 17.0, 'coverage_true_min_p90_NW': 48.0, 'coverage_true_rate_below_threshold_NW': 0.9953, 'specificity_pred_min_mean_NW': 15.999245453859503, 'specificity_pred_min_median_NW': 13.0, 'specificity_pred_min_p90_NW': 32.0, 'specificity_pred_rate_below_threshold_NW': 0.9975854523504112, 'pairs_within_threshold_NW': 7543, 'precision_at_threshold_NW': 0.5691541537765035, 'recall_at_threshold_NW': 0.7543, 'f1_at_threshold_NW': 0.6487765019567368}

## $K=2$

[COMP-F:2] comp 5258/5258 done | cumul points labellisés=128920 | conflits=21392 | RSS=14.7 GiB
Clusters finaux : 13053 | bruit : 140789 | RSS=14.7 GiB

{'n_true_centers': 10000, 'n_pred_centers': 13053, 'classes_retrouvees': 7378, 'precision_classes': 0.565234045813223, 'recall_classes': 0.7378, 'f1_classes': 0.6400902268685205, 'classes_en_trop': 5675, 'avg_editdist_on_matched_NW': 17.950799674708595, 'median_editdist_on_matched_NW': 16.0, 'p90_editdist_on_matched_NW': 35.0, 'max_editdist_on_matched_NW': 50.0, 'exact_match_rate_on_matched_NW': 0.0035239902412577935, 'count_dist_0_NW': 26, 'count_dist_1_NW': 58, 'count_dist_2_NW': 143, 'count_dist_3_NW': 142, 'avg_editdist_on_matched_HW': 15.358362699918677, 'median_editdist_on_matched_HW': 13.0, 'p90_editdist_on_matched_HW': 30.0, 'max_editdist_on_matched_HW': 49.0, 'exact_match_rate_on_matched_HW': 0.0035239902412577935, 'count_dist_0_HW': 26, 'count_dist_1_HW': 82, 'count_dist_2_HW': 162, 'count_dist_3_HW': 199, 'coverage_true_min_mean_NW': 24.5173, 'coverage_true_min_median_NW': 23.0, 'coverage_true_min_p90_NW': 49.0, 'coverage_true_rate_below_threshold_NW': 0.9872, 'specificity_pred_min_mean_NW': 17.785796368650885, 'specificity_pred_min_median_NW': 15.0, 'specificity_pred_min_p90_NW': 35.0, 'specificity_pred_rate_below_threshold_NW': 0.9942541944380602, 'pairs_within_threshold_NW': 7378, 'precision_at_threshold_NW': 0.565234045813223, 'recall_at_threshold_NW': 0.7378, 'f1_at_threshold_NW': 0.6400902268685205}


## $K = 1$

[COMP-F:2] comp 3724/3724 done | cumul points labellisés=181699 | conflits=0 | RSS=8.7 GiB
Clusters finaux : 7650 | bruit : 88014 | RSS=8.7 GiB


{'n_true_centers': 10000, 'n_pred_centers': 7650, 'classes_retrouvees': 7114, 'precision_classes': 0.9299346405228758, 'recall_classes': 0.7114, 'f1_classes': 0.8061189801699717, 'classes_en_trop': 536, 'avg_editdist_on_matched_NW': 24.56452066348046, 'median_editdist_on_matched_NW': 24.0, 'p90_editdist_on_matched_NW': 39.0, 'max_editdist_on_matched_NW': 50.0, 'exact_match_rate_on_matched_NW': 0.0014056789429294349, 'count_dist_0_NW': 10, 'count_dist_1_NW': 10, 'count_dist_2_NW': 26, 'count_dist_3_NW': 9, 'avg_editdist_on_matched_HW': 21.201433792521787, 'median_editdist_on_matched_HW': 21.0, 'p90_editdist_on_matched_HW': 33.0, 'max_editdist_on_matched_HW': 50.0, 'exact_match_rate_on_matched_HW': 0.0014056789429294349, 'count_dist_0_HW': 10, 'count_dist_1_HW': 22, 'count_dist_2_HW': 23, 'count_dist_3_HW': 24, 'coverage_true_min_mean_NW': 30.5867, 'coverage_true_min_median_NW': 29.0, 'coverage_true_min_p90_NW': 50.0, 'coverage_true_rate_below_threshold_NW': 0.9244, 'specificity_pred_min_mean_NW': 25.231503267973856, 'specificity_pred_min_median_NW': 24.0, 'specificity_pred_min_p90_NW': 40.0, 'specificity_pred_rate_below_threshold_NW': 0.989281045751634, 'pairs_within_threshold_NW': 7114, 'precision_at_threshold_NW': 0.9299346405228758, 'recall_at_threshold_NW': 0.7114, 'f1_at_threshold_NW': 0.8061189801699717}



# LSH_threshold = 0.35

# $K = 1$

{'n_true_centers': 10000, 'n_pred_centers': 9323, 'classes_retrouvees': 9071, 'precision_classes': 0.9729700740105116, 'recall_classes': 0.9071, 'f1_classes': 0.9388811261191327, 'classes_en_trop': 252, 'avg_editdist_on_matched_NW': 24.233822070334032, 'median_editdist_on_matched_NW': 24.0, 'p90_editdist_on_matched_NW': 35.0, 'max_editdist_on_matched_NW': 50.0, 'exact_match_rate_on_matched_NW': 0.0014331385734759122, 'count_dist_0_NW': 13, 'count_dist_1_NW': 5, 'count_dist_2_NW': 28, 'count_dist_3_NW': 10, 'avg_editdist_on_matched_HW': 21.420901774887003, 'median_editdist_on_matched_HW': 21.0, 'p90_editdist_on_matched_HW': 31.0, 'max_editdist_on_matched_HW': 49.0, 'exact_match_rate_on_matched_HW': 0.0014331385734759122, 'count_dist_0_HW': 13, 'count_dist_1_HW': 17, 'count_dist_2_HW': 27, 'count_dist_3_HW': 15, 'coverage_true_min_mean_NW': 26.099, 'coverage_true_min_median_NW': 25.0, 'coverage_true_min_p90_NW': 39.0, 'coverage_true_rate_below_threshold_NW': 0.9761, 'specificity_pred_min_mean_NW': 24.3795988415746, 'specificity_pred_min_median_NW': 24.0, 'specificity_pred_min_p90_NW': 35.0, 'specificity_pred_rate_below_threshold_NW': 0.9960313203904323, 'pairs_within_threshold_NW': 9071, 'precision_at_threshold_NW': 0.9729700740105116, 'recall_at_threshold_NW': 0.9071, 'f1_at_threshold_NW': 0.9388811261191327}


# $K=2$
{'n_true_centers': 10000, 'n_pred_centers': 13977, 'classes_retrouvees': 8306, 'precision_classes': 0.594262001860199, 'recall_classes': 0.8306, 'f1_classes': 0.6928306293531301, 'classes_en_trop': 5671, 'avg_editdist_on_matched_NW': 19.957861786660246, 'median_editdist_on_matched_NW': 20.0, 'p90_editdist_on_matched_NW': 34.0, 'max_editdist_on_matched_NW': 50.0, 'exact_match_rate_on_matched_NW': 0.003009872381411028, 'count_dist_0_NW': 25, 'count_dist_1_NW': 40, 'count_dist_2_NW': 96, 'count_dist_3_NW': 79, 'avg_editdist_on_matched_HW': 17.469058511919094, 'median_editdist_on_matched_HW': 17.0, 'p90_editdist_on_matched_HW': 30.0, 'max_editdist_on_matched_HW': 49.0, 'exact_match_rate_on_matched_HW': 0.003009872381411028, 'count_dist_0_HW': 25, 'count_dist_1_HW': 60, 'count_dist_2_HW': 109, 'count_dist_3_HW': 100, 'coverage_true_min_mean_NW': 24.0554, 'coverage_true_min_median_NW': 23.0, 'coverage_true_min_p90_NW': 48.0, 'coverage_true_rate_below_threshold_NW': 0.9902, 'specificity_pred_min_mean_NW': 18.73821277813551, 'specificity_pred_min_median_NW': 17.0, 'specificity_pred_min_p90_NW': 33.0, 'specificity_pred_rate_below_threshold_NW': 0.9944909494168992, 'pairs_within_threshold_NW': 8306, 'precision_at_threshold_NW': 0.594262001860199, 'recall_at_threshold_NW': 0.8306, 'f1_at_threshold_NW': 0.6928306293531301}
